In [4]:
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [16]:
df_completo = pd.read_parquet(r'dataset_completo.parquet')
df_director = pd.read_parquet(r'director_dataset.parquet')
df_actor = pd.read_parquet(r'actor_dataset.parquet')
df_scrore = pd.read_parquet(r'score_titulo.parquet')
df_votos = pd.read_parquet(r'votos_titulo.parquet')
df_filmaciones = pd.read_parquet(r'filmaciones.parquet')

In [13]:
import unidecode
import pandas as pd

# Función para limpiar nombres
def limpiar_texto_completo(texto):
    if pd.isnull(texto):
        print(f"Valor nulo encontrado: {texto}")
        return None    
    try:
        texto = str(texto).lower().strip()  # Asegurarse de que el texto es una cadena
        texto = ' '.join(texto.split())  # Eliminar espacios extra
        texto = unidecode.unidecode(texto)  # Eliminar caracteres especiales
    except Exception as e:
        print(f"Error procesando el texto: {texto}, Error: {e}")
        return None 
    return texto

# Aplicar la limpieza a los nombres de actores y directores
df_actor['actor_name'] = df_actor['actor_name'].apply(limpiar_texto_completo)
df_director['director_name'] = df_director['director_name'].apply(limpiar_texto_completo)

# Verificar los primeros registros después de la limpieza
print("Primeros 5 actores después de la limpieza:\n", df_actor['actor_name'].head(10))
print("Primeros 5 directores después de la limpieza:\n", df_director['director_name'].head(10))

# Verificar si puedes encontrar un actor específico después de la limpieza
actor_buscado = limpiar_texto_completo('laurence fishburne')

# Verificar si el nombre del actor es válido antes de buscar
if actor_buscado is not None:
    actor_films = df_actor[df_actor['actor_name'].str.contains(actor_buscado, case=False, na=False)]
    print(f"Películas encontradas para el actor '{actor_buscado}':\n", actor_films[['title', 'actor_name']])
else:
    print("El nombre del actor no es válido.")

# Verificar si puedes encontrar un director específico después de la limpieza
director_buscado = limpiar_texto_completo('oliver parker')

# Verificar si el nombre del director es válido antes de buscar
if director_buscado is not None:
    director_films = df_director[df_director['director_name'].str.contains(director_buscado, case=False, na=False)]
    print(f"Películas encontradas para el director '{director_buscado}':\n", director_films[['title', 'director_name']])
else: 
    print("El nombre del director no es válido.")

Primeros 5 actores después de la limpieza:
 0    laurence fishburne
1           irene jacob
2       kenneth branagh
3      nathaniel parker
4             indra ove
5       michael maloney
6          anna patrick
7      nicholas farrell
8         michael sheen
9          ian mckellen
Name: actor_name, dtype: object
Primeros 5 directores después de la limpieza:
 0        oliver parker
1        oliver parker
2        oliver parker
3        oliver parker
4        oliver parker
5        oliver parker
6        oliver parker
7        oliver parker
8        oliver parker
9    richard loncraine
Name: director_name, dtype: object
Películas encontradas para el actor 'laurence fishburne':
      title          actor_name
0  othello  laurence fishburne
Películas encontradas para el director 'oliver parker':
                  title  director_name
0              othello  oliver parker
1              othello  oliver parker
2              othello  oliver parker
3              othello  oliver parker
4   

In [14]:
def recomendacion(titulo, n_recomendaciones=5):
    # Verificar si el título está en el dataset
    if titulo not in df_completo['title'].values:
        return f"El título '{titulo}' no se encuentra en el dataset."

    # Obtener el índice de la película en el dataset
    idx = df_completo[df_completo['title'] == titulo].index[0]
    
    # Calcular las similitudes utilizando la matriz TF-IDF (o usando las columnas numéricas normalizadas)
    cosine_similarities = cosine_similarity([features_matrix[idx]], features_matrix).flatten()
    
    # Obtener los índices de las películas más similares
    similar_indices = cosine_similarities.argsort()[::-1][1:n_recomendaciones + 1]
    
    # Devolver los títulos de las películas recomendadas
    return df_completo['title'].iloc[similar_indices].tolist()


In [8]:
# Guardar el dataset en formato Parquet
df_completo.to_parquet('dataset_preprocesado.parquet')

In [9]:
'''# Crear el DataFrame de actores (ignorando películas con presupuesto = 0)
df_actores = df_datos[['actor_name', 'title', 'vote_average', 'revenue', 'budget']].dropna()
df_actores = df_actores[df_actores['budget'] > 0]  # Ignorar películas con presupuesto 0

# Calcular el retorno solo para películas con presupuesto > 0
df_actores['return'] = df_actores.apply(lambda row: row['revenue'] / row['budget'], axis=1)

# Crear un DataFrame que agrupe por actor_name, calcule la cantidad de películas y el promedio de retorno
df_actores_agrupado = df_actores.groupby('actor_name').agg(
    cantidad_peliculas=('title', 'count'),
    retorno_total=('return', 'sum'),
    retorno_promedio=('return', 'mean')
).reset_index()

# Guardar el dataset en formato Parquet
df_actores_agrupado.to_parquet('dataset_actores.parquet')

In [10]:
'''# Crear el DataFrame de directores (ignorando películas con presupuesto = 0)
df_directores = df_datos[['director_name', 'title', 'release_date', 'revenue', 'budget']].dropna()
df_directores = df_directores[df_directores['budget'] > 0]  # Ignorar películas con presupuesto 0

# Calcular el retorno individual solo para películas con presupuesto > 0
df_directores['return'] = df_directores.apply(lambda row: row['revenue'] / row['budget'], axis=1)

# Guardar el DataFrame de directores sin agrupar para mostrar las películas individuales
df_directores_individual = df_directores[['director_name', 'title', 'release_date', 'return', 'budget', 'revenue']]

# Guardar el dataset en formato Parquet
df_directores_individual.to_parquet('dataset_directores.parquet')

In [11]:
'''# Verifica las primeras filas de df_actores y df_directores
print(df_actores_agrupado.head())
print(df_directores_individual.head())

# Verifica la información general de los DataFrames, como tipos de datos y valores nulos
print(df_actores_agrupado.info())
print(df_directores_individual.info())

# Verifica si hay valores nulos en columnas importantes como 'budget' y 'revenue'
print(df_actores_agrupado[['budget', 'revenue']].isnull().sum())
print(df_directores_individual[['budget', 'revenue']].isnull().sum())

# Verifica estadísticas descriptivas, especialmente en 'budget' y 'revenue'
print(df_actores_agrupado[['budget', 'revenue']].describe())
print(df_directores_individual[['budget', 'revenue']].describe())


      actor_name  cantidad_peliculas  retorno_total  retorno_promedio
0        50 cent                   1       1.469629          1.469629
1   a. j. langer                   1       0.000000          0.000000
2   a.j. buckley                   1       0.464000          0.464000
3   a.j. johnson                   1      10.554400         10.554400
4  aaron eckhart                   1       0.942150          0.942150
   director_name     title release_date    return    budget   revenue
88     spike lee  clockers   1995-09-15  0.522861  25000000  13071518
89     spike lee  clockers   1995-09-15  0.522861  25000000  13071518
90     spike lee  clockers   1995-09-15  0.522861  25000000  13071518
91     spike lee  clockers   1995-09-15  0.522861  25000000  13071518
92     spike lee  clockers   1995-09-15  0.522861  25000000  13071518
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4431 entries, 0 to 4430
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  

KeyError: "None of [Index(['budget', 'revenue'], dtype='object')] are in the [columns]"

,title,movie_id,release_date,vote_count,vote_average,actor_name,director_name,revenue,budget
0,othello,16420,1995-12-15,33.0,7.0,laurence fishburne,oliver parker,0,0
1,othello,16420,1995-12-15,33.0,7.0,irene jacob,oliver parker,0,0
2,othello,16420,1995-12-15,33.0,7.0,kenneth branagh,oliver parker,0,0
3,othello,16420,1995-12-15,33.0,7.0,nathaniel parker,oliver parker,0,0
4,othello,16420,1995-12-15,33.0,7.0,indra ove,oliver parker,0,0
...,...,...,...,...,...,...,...,...,...
35388,the fat and lean wrestling match,106807,1900-01-01,6.0,6.5,georges melies,georges melies,0,0
35389,st. michael had a rooster,67179,1972-01-01,3.0,6.0,giulio brogi,paolo taviani,0,0
35390,st. michael had a rooster,67179,1972-01-01,3.0,6.0,renato cestie,paolo taviani,0,0
35391,st. michael had a rooster,67179,1972-01-01,3.0,6.0,vito cipolla,paolo taviani,0,0
